In [ ]:
from connectivity import plot_graph_circos
import networkx as nx

In [ ]:
from neurolib.utils.loadData import Dataset
ds = Dataset("gw")
G = nx.from_numpy_matrix(ds.Cmats[1])

In [ ]:
plot_graph_circos(G)